In [1]:
import audiolabel
import parselmouth as ps  # This provides Praat functionality within python
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as LA  # This provides linear algebra helper functions for calculating distances, which we use in the function apPal

pd.options.mode.chained_assignment = None  # default='warn'; this line suppresses some warnings from pandas

# Unit 3: XRMB

The X-Ray Microbeam Database is a set of articulatory and acoustic recordings collected in 1989-1991 at the University of Wisconsin

We have two types of raw data available -- audio recordings of a variety of speech tasks taken on by speakers in the study (**audio**) and the positions of pellets glued to different parts of speakers' jaw, tongue, lips, and face during these speech tasks (**xrmb**). We also have access to annotations in the form of TextGrids generated using forced alignment by Keith Johnson for a previous course. 
These data have the following file structure and naming conventions:
- data
    - subject -- JW followed by a number
        - PAL.DAT -- the palate trace for the subject
        - PHA.DAT -- the approximated pharyngeal trace for the subject
        - taN.wav -- audio recordings without accompanying xrmb data
        - tpN.wav -- audio recordings *with* accompanying xrmb data
        - tpN.txy -- accompanying xrmb data
- annotation
    - subject -- same as above
        - tpN.TextGrid -- accompanying forced aligned annotations; these have the typical forced aligned structure of words on tier 1 and phones on tier 2

In [2]:
annotations = os.path.abspath('./annotation')
datadir = os.path.abspath('./data')

## Getting to know the xrmb data

Let's check out some data and see what we're working with! First we can establish a list of the subjects, and then pick one of them and look at their files.

In [3]:
subjects = [s for s in os.listdir(datadir) if os.path.isdir(os.path.join(datadir,s))]
len(subjects) # how many subjects are there?

48

## Manipulating the xrmb data

There are many different kinds of questions we might try to answer with these data. The specific question we are trying to replicate is (in two parts)
1. Are tongue tip constrictions in post-vocalic /l/s weaker preceding bilabial and velar stops compared to alveolar stops?
2. Is this difference between magnitude of constriction affected by lexical frequency whereby more frequent words show weaker constrictions than stronger words?

One manipulation we'll need is to measure the distance between the tongue tip (T1) and the palate trace, aka "tongue tip aperture". We'll find all of the distances from the T1 point to every point on the palate and give the minimum of those points, as well as the spot on the palate resulting in that minimum.
(Another, almost certaily faster, way to approach this same question is to use the y-cooridinate of T1 as a proxy for tongue height.)

The following is a function that determines the distance between the palate and any tongue point using the method above. 
- It takes as input the **row** of data from the xrmb dataframe, the dataframe containing the palate trace **paldf**, and the number of the point to measure distance from **TN**.
- It returns the distance between that point and the palate trace

In [4]:
def apPal(row, paldf, TN):
    
    xval=row[''.join(['T',str(TN),'x'])]
    yval=row[''.join(['T',str(TN),'y'])]
    paltemp = paldf.copy()
    paltemp['ap']=paltemp.apply(lambda x: LA.norm(np.array([xval,yval])-np.array([x.x, x.y])), axis=1)
    
    return paltemp.iloc[paltemp['ap'].argmin()]

In [5]:
names=[
    'time', 'ULx', 'ULy', 'LLx', 'LLy', 'T1x', 'T1y', 'T2x', 'T2y',
    'T3x', 'T3y', 'T4x', 'T4y', 'MIx', 'MIy', 'MMx', 'MMy'
]

**This is the cell where you'll make most changes for gathering data.**

In [12]:
alldf = pd.DataFrame()

# for sub in subjects[0:1]: ## you can uncomment this if you want to test just a single subject before running it on everyone
for sub in subjects:
    
    subdata = os.path.join(datadir,sub)
    subannotations = os.path.join(annotations,sub)
    xfiles = [f for f in os.listdir(subdata) if f[-4:]=='.txy']  # These are the xray files
    tgfiles = [f for f in os.listdir(subannotations) if f[-9:]=='.TextGrid']  # We don't really need to check if they are TextGrids but it's still nice to

    palfile = os.path.join(subdata, 'PAL.DAT')
    phafile = os.path.join(subdata, 'PHA.DAT')
    paldf = pd.read_table(palfile, sep='\s+', header=None, names=['x', 'y'])/1000
    phadf = pd.read_table(phafile, sep='\s+', header=None, names=['x', 'y'])/1000
    
    for f in xfiles:
        prefix = f[:-4]
        try:
            xdata = pd.read_table(os.path.join(subdata,prefix+'.txy'),names=names)

            xdata = xdata.replace(1000000,np.nan)
            xdata = xdata/1000
            xdata.time = xdata.time/1000

            if not os.path.isfile(os.path.join(annotations,sub,prefix+'.TextGrid')):
                print(' '.join(["No annotation found for",sub,prefix]))
                continue

            tg = audiolabel.LabelManager(from_file=os.path.join(annotations,sub,prefix+'.TextGrid'),from_type='praat')
            [phonedf, worddf] = tg.as_df()

            phonedf=phonedf.rename(columns={"text":"phone"})
            phonedf['phoneprev'] = phonedf.phone.shift()
            phonedf['phonenext'] = phonedf.phone.shift(-1)
            worddf=worddf.rename(columns={"text":"word"})
            timesdf = pd.merge_asof(phonedf, worddf, on='t1')
            timesdf=timesdf.rename(columns={"t2_x":"t2"})
            timesdf=timesdf.rename(columns={"center_x":"center"})
            timesdf=timesdf[['t1','t2','center','phone','phoneprev','phonenext','word']]
            timesdf['wordonset'] = ~(timesdf.word==timesdf.word.shift())
            timesdf['wordend'] = ~(timesdf.word==timesdf.word.shift(-1))
            timesdf['vowel'] = timesdf.phone.apply(lambda x: len(x)==3)
            timesdf['stress'] = timesdf.phone.apply(lambda x: x[-1] if len(x)>0 else '')
            timesdf = timesdf.replace(np.nan,'')

            # xdata = pd.merge_asof(xdata, timesdf, left_on='time', right_on='t1')
            xdata = pd.merge_asof(timesdf, xdata, left_on='center', right_on='time')  ## this will get just the xrmb data at the midpoint of each phone

            # edit the next line for the segment(s) you're interested in
            segdf = xdata[xdata.vowel==True]
            segdf['file']=prefix
            segdf['subject']=sub

            if len(segdf)>0:  # edit these two lines for the points you are interested in
                alldf = pd.concat([alldf,segdf])
        except:
            print(' '.join(["Error for",sub,prefix]))
    
print('Done!')

No annotation found for JW21 tp117
No annotation found for JW21 tp118_2
No annotation found for JW21 tp106
No annotation found for JW21 tp118
No annotation found for JW19 tp117
No annotation found for JW19 tp106
No annotation found for JW19 tp118
No annotation found for JW502 tp106_3
No annotation found for JW502 tp106_2
No annotation found for JW502 tp116
No annotation found for JW502 tp117
No annotation found for JW502 tp115
No annotation found for JW502 tp114
No annotation found for JW502 tp110
No annotation found for JW502 tp111
No annotation found for JW502 tp107
No annotation found for JW502 tp113
No annotation found for JW502 tp112
No annotation found for JW502 tp106
No annotation found for JW502 tp108
No annotation found for JW502 tp109
No annotation found for JW502 tp118
No annotation found for JW502 tp117_2
No annotation found for JW26 tp116
No annotation found for JW26 tp117
No annotation found for JW26 tp115
No annotation found for JW26 tp114
No annotation found for JW26 tp

This dataset looks a bit different from thes ones we used before.

In [13]:
alldf.head()

,t1,t2,center,phone,phoneprev,phonenext,word,wordonset,wordend,vowel,...,T3x,T3y,T4x,T4y,MIx,MIy,MMx,MMy,file,subject
2,1.149887,1.209751,1.179819,EH1,W,N,WHEN,False,False,True,...,-41.848,9.468,-53.011,4.123,-5.018,-9.308,-46.040,-7.177,tp029,JW21
4,1.259637,1.509070,1.384354,AO1,N,L,ALL,True,False,True,...,-44.835,7.001,-60.768,7.890,-4.288,-17.083,-44.974,-12.846,tp029,JW21
6,1.539002,1.658730,1.598866,EH1,L,L,ELSE,True,False,True,...,-37.528,9.365,-52.774,8.219,-5.861,-14.743,-46.428,-11.020,tp029,JW21
10,2.031795,2.187528,2.109662,EY1,F,L,FAILS,False,False,True,...,-33.315,11.986,-46.779,8.145,-4.688,-12.919,-45.166,-10.130,tp029,JW21
15,2.915873,3.045578,2.980726,UW1,Y,S,USE,False,False,True,...,-37.267,13.433,-50.724,8.504,-2.048,-4.914,-43.646,-4.789,tp029,JW21


Squish all vowels of the same quality together (regardless of stress) and find their mean values.

In [16]:
alldf['vowelID']=alldf.phone.apply(lambda x: x[0:2])
summarydf = alldf.groupby(['subject','word','vowelID']).mean().reset_index()
summarydf.head()

KeyError: 'vowelID'

Let's do a quick check to see what we're working with and verify that things are working (mostly) as expected.

In [ ]:
summarydf.vowel.unique()

In [ ]:
sns.boxplot(x='vowelID',y='T3y',data=summarydf)

In [ ]:
sns.boxplot(x='vowelID',y='T3x',data=summarydf)

We can create a new function that will give us the area inside any polygon.
- input: an unordered list of ordered (x,y) coordinates)
- output: the area of the polygon

In [ ]:
def PolygonArea(corners):
    n = len(corners)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

Add word frequency information now.

In [ ]:
freqdf = pd.read_table('./SUBTLEXus74286wordstextversion.txt')
freqdf.head(10)

We need to join the frequency and l dataframes, but we have one more problem to handle -- the words from the XRMB database are in all capital letters while the ones in SUBTLEX are primarily lowercase but with proper names and acronyms capitalized.
We can handle this by creating a new column in _both_ dataframes called 'wordlc' which is equivalent to the 'word' columns, but only lowercase letters.

In [ ]:
freqdf['wordlc']=freqdf.Word.apply(lambda x: x.lower() if type(x)==str else x)
summarydf['wordlc']=summarydf.word.apply(lambda x: x.lower())
summarydf.head()

Now we can merge the dataframes on the newly created column.

In [ ]:
summarydf = summarydf.merge(freqdf,left_on='wordlc',right_on='wordlc')
summarydf.head()

In [ ]:
summarydf['freqcat']=np.floor(summarydf.Lg10WF)
summarysummarydf = summarydf.groupby(['subject','vowel','freqcat']).mean().reset_index()
summarysummarydf = summarysummarydf[['subject','vowel','freqcat','ULx','ULy','T3x','T3y']]
summarysummarydf = summarysummarydf[summarysummarydf.vowel.isin(['IY','UW','AA','AE'])]
summarysummarydf.head()

In [ ]:
byfreqdf = summarysummarydf.pivot(index=['subject','freqcat'], columns=['vowel'], values=['T3x','T3y'])

In [ ]:
byfreqdf['area']=byfreqdf.apply(lambda x: PolygonArea([(x.T3x.AA,x.T3y.AA),(x.T3x.IY,x.T3y.IY),
                                      (x.T3x.UW,x.T3y.UW),(x.T3x.AE,x.T3y.AE)]),axis=1)
byfreqdf

In [ ]:
sns.boxplot(x='freqcat', y='area', data=byfreqdf.reset_index())
plt.ylim(0,20)

What do we see? Do we think there is much of a relationship between word frequency (higher numbers refer to more frequent words) and vowel polygon area (higher numbers means larger area and more peripherality)?